In [11]:
import tweepy
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv()

True

In [13]:
CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")
ACCESS_KEY = os.getenv("ACCESS_KEY")
ACCESS_SECRET = os.getenv("ACCESS_SECRET")

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print('Authentication OK')
except:
    print('Error during authentication')

In [46]:
## Twitter bot will search for mentions

def get_mentions():
    return api.mentions_timeline()

mentions = get_mentions()

In [47]:
for x in mentions:
    print(x)

Status(_api=<tweepy.api.API object at 0x00000219868C1AC0>, _json={'created_at': 'Mon Jun 14 22:57:52 +0000 2021', 'id': 1404573832577495041, 'id_str': '1404573832577495041', 'text': '@NathanFleck8 YO', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'NathanFleck8', 'name': 'Nathan Fleck', 'id': 1372271424241602564, 'id_str': '1372271424241602564', 'indices': [0, 13]}], 'urls': []}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': 1404549544537464834, 'in_reply_to_status_id_str': '1404549544537464834', 'in_reply_to_user_id': 1372271424241602564, 'in_reply_to_user_id_str': '1372271424241602564', 'in_reply_to_screen_name': 'NathanFleck8', 'user': {'id': 181271196, 'id_str': '181271196', 'name': 'Peje', 'screen_name': 'Pejegordo', 'location': 'Puerto Rico', 'description': 'Mechanical Engineer. Into: fishing, growing, evolution, decentralization, human behavior, information distrib

## Twitter bot will search for replies to a certain tweet

In [6]:
def get_tweet_comments(input_tweet):

    """

    Args:
        input_tweet: A Twitter API response object

    Returns: A list of the text for all found replies.

    """
    comments = []
    name = input_tweet._json['user']['screen_name']
    tweet_id = input_tweet._json['id']
    for tweet in tweepy.Cursor(api.search,q=f"to:{name}", result_type = 'recent',tweet_mode = 'extended').items(100):
        if hasattr(tweet, 'in_reply_to_status_id_str'):
            if str(tweet.in_reply_to_status_id_str) == str(tweet_id):
                comments.append(tweet._json['full_text'])
                return tweet
    return comments


Status(_api=<tweepy.api.API object at 0x00000173C35AE8B0>, _json={'created_at': 'Thu Jun 10 16:47:13 +0000 2021', 'id': 1403031007406788612, 'id_str': '1403031007406788612', 'full_text': '@lmakjgb https://t.co/gIbSFosA8J', 'truncated': False, 'display_text_range': [8, 8], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'lmakjgb', 'name': 'covered.', 'id': 1241928735068246018, 'id_str': '1241928735068246018', 'indices': [0, 8]}], 'urls': [], 'media': [{'id': 1403031001782173698, 'id_str': '1403031001782173698', 'indices': [9, 32], 'media_url': 'http://pbs.twimg.com/media/E3iRQzYWQAIo6Ym.jpg', 'media_url_https': 'https://pbs.twimg.com/media/E3iRQzYWQAIo6Ym.jpg', 'url': 'https://t.co/gIbSFosA8J', 'display_url': 'pic.twitter.com/gIbSFosA8J', 'expanded_url': 'https://twitter.com/KCShade1/status/1403031007406788612/photo/1', 'type': 'photo', 'sizes': {'small': {'w': 680, 'h': 644, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w':

## Twitter bot will reply to the tweet

In [97]:
def twitter_reply(status_id, text):
    if text == "":
        reply_text = '''
        ------- test -------
        We noticed that you are reporting a police incident.
        '''
    else:
        reply_text = text

    tweet = api.update_status(status = reply_text, in_reply_to_status_id=status_id,tweet_mode = 'extended')
    return tweet

#twitter_reply(1395143654331744256)

In [12]:
df_data = []
for tweet in api.search(q='police', lang='en', rpp=5)[:5]:
    test_data = [tweet.user.screen_name, tweet.id, tweet.text]
    df_data.append(test_data)
df = pd.DataFrame(data=df_data, columns=['username', 'id', 'tweet'])
df

,username,id,tweet
0,ksmoola1,1402755452027523075,@patriotcanuck65 Not a gun in sight except fro...
1,NJ_Bronco_Fan,1402755451129942019,RT @MacFarlaneNews: FLASH: Georgia roofing co...
2,oneiriclillies,1402755447891939332,RT @hotties4dwt: what's happening in Venezuela...
3,Alejand14092042,1402755446444789771,@DailyYaeContent Dunno... if you kissed her......
4,victor2exl,1402755445257760773,RT @MaziNnamdiKanu: It’s pathetic that Nigeria...


## Twitter bot will reply to the user through DM

In [80]:
def direct_message(username):
    user = api.get_user(username)
    message = '''
    Just testing some features.
    This sends a direct message to the person
    '''
    api.send_direct_message(user.id, message)
    return 'direct message sent'

direct_message('@JoanVillar14')

'direct message sent'

Improvements:

1. Depending on the reply, gather more information such as:
- did it really happen?
- where did it happen

2. They pin the place of incident on a map
- If tweet's geolocation is on, get the coordinates

-------


* There are a few things to keep in mind in designing a model that replies to specific replies. This might be another project in the future.
* Since the incidents are already identified, it might be best to just get the location from the user.

## The original tweet id will contain the tweet object

## The bot initial response tweet id will store the tweet id for the original tweet

## When someone responds to the bot, store the tweet id as key and use the responded to id as value

The user: NathanFleck8 said: test tweet
The user: NathanFleck8 said: Here you go
